In [1]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.09.26 데이터 추출 일부 실패 - 31140 데이터가 대부분 없음, InterfaceError: Python type ResultSet cannot be converted
cursor = mydb.cursor(prepared=True)
citycode = '32040'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)

now = datetime.now()
nowDate = now.strftime('%Y%m%d')

b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        if rows == []:
            print("location {} | {} 데이터가 없습니다.".format(location[0],join_data))
            cnt += 1
            continue
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')                
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            if nodeno == []:
                nodeno = None
            else:
                nodeno = nodeno[0].text
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')  
            updowncd = item.findAll('updowncd')
            
            if updowncd == []:
                updowncd = None
            elif updowncd[0].text == '0':
                updowncd = '상행'
            elif updowncd[0].text == '1':
                updowncd = '하행'

            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd, citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno, nodeord[0].text, routeid[0].text, updowncd, location[0])

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20210926 | location : 32040 | cnt : 1 | start TSB253000001 public_busroute_info
조회날짜 : 20210926 | location : 32040 | cnt : 2 | start TSB253000002 public_busroute_info
조회날짜 : 20210926 | location : 32040 | cnt : 3 | start TSB253000003 public_busroute_info
조회날짜 : 20210926 | location : 32040 | cnt : 4 | start TSB253000004 public_busroute_info
조회날짜 : 20210926 | location : 32040 | cnt : 5 | start TSB253000005 public_busroute_info


InterfaceError: Python type ResultSet cannot be converted